In [22]:
import os
import sys
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import shutil
import glob
import json
from PIL import Image,ImageOps
from time import time
import random
import zipfile

In [23]:
#shutil.rmtree(os.path.join(os.getcwd(),'Images')) # delete entire directory

In [24]:
zip = zipfile.ZipFile('task1_videos_mp4.zip')
zip.extractall()

In [25]:
zip = zipfile.ZipFile('task1_classic_classification.zip')
zip.extractall()

In [7]:
path = os.path.join(os.getcwd(),'Images/train') 
os.makedirs(path)

In [8]:
path = os.path.join(os.getcwd(),'Images/test') 
os.makedirs(path)

In [9]:
def videos_dir_sort(rpath):
    vids=[]
    for i in os.listdir(rpath):
        vids.append(i)
    vids.sort()
    return vids
    

In [10]:
def frame_sort(frms,tp):
    fn=[]
    for i in range(len(tp)):
        for j in range(len(frms)):
            if int(tp[i]) == int(frms[j].split('.')[0].split('_')[2]):
                fn.insert(i,frms[j])
            else:
                continue
    return fn

In [11]:
def video_process(rpath,wpath,ch,ln):
  imgs,vids=[],[]
  vpath = videos_dir_sort(rpath)
  vpath = vpath[0:ln]
  for i in vpath:
    vid = cv2.VideoCapture(rpath+'/'+i)
    vifn = 'task1/'+ch+'/'+i.split('.')[0].split('_')[0]+'_task1_annotator1'
    vids.append(vifn)
    fram = 1
    frms,temp=[],[]
    while(True):
      ret,frame = vid.read()
      if ret:
           ifn = i.split('.')[0].split('_')[0]+'_frameno_'+str(fram)+'.jpg'
           fn = wpath+'/'+ifn
           frms.append(ifn)
           frame = cv2.resize(frame,(600,330)) # 570*1024
           temp.append(fram)
           cv2.imwrite(fn, frame)
           fram += 1
      else:
        break
    vid.release()
    cv2.destroyAllWindows()
    temp.sort()
    frmes = frame_sort(frms,temp)
    imgs.append((np.array(frmes)))
    print("Video frames for the video {} is processed".format(i))
  print("All video frame have been processed sucessfully!")
  return imgs,vids

In [12]:
def json_file(ch):
  ifn = os.path.join(os.getcwd(),'task1_classic_classification')
  if ch=='train':
    fn = ifn+'/'+'calms21_task1_train.json'
    ln= int(18)
  else:
    fn = ifn+'/'+'calms21_task1_test.json'
    ln= int(2)
  return fn , ln

In [13]:
def json_file_process(fn,vids):
  bhv,locs=[],[]
  with open(fn) as files:
    file_contents = files.read()
  parsed_json = json.loads(file_contents)
  temp = parsed_json['annotator-id_0']
  for i in vids:
    tmp = temp['{}'.format(i)]
    bhv.append(tmp['annotations'])
    locs.append(tmp['keypoints'])
  bv = np.concatenate(bhv, axis=0)
  lcs = np.concatenate(locs, axis=0)
  return bv,lcs

In [14]:
def parameter_extraction(lcs):
  m1x1,m1x2,m1x3,m1x4,m1x5,m1x6,m1x7,m2x1,m2x2,m2x3,m2x4,m2x5,m2x6,m2x7 = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
  m1y1,m1y2,m1y3,m1y4,m1y5,m1y6,m1y7,m2y1,m2y2,m2y3,m2y4,m2y5,m2y6,m2y7 = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
  for z in range(len(lcs)):
    m1x1.append(round((lcs[z][0][0][0]/1024)*600))
    m2x1.append(round((lcs[z][1][0][0]/1024)*600))
    m1y1.append(round((lcs[z][0][1][0]/1024)*600))
    m2y1.append(round((lcs[z][1][1][0]/1024)*600))
    m1x2.append(round((lcs[z][0][0][1]/1024)*600))
    m2x2.append(round((lcs[z][1][0][1]/1024)*600))
    m1y2.append(round((lcs[z][0][1][1]/1024)*600))
    m2y2.append(round((lcs[z][1][1][1]/1024)*600))
    m1x3.append(round((lcs[z][0][0][2]/1024)*600))
    m2x3.append(round((lcs[z][1][0][2]/1024)*600))
    m1y3.append(round((lcs[z][0][1][2]/1024)*600))
    m2y3.append(round((lcs[z][1][1][2]/1024)*600))
    m1x4.append(round((lcs[z][0][0][3]/1024)*600))
    m2x4.append(round((lcs[z][1][0][3]/1024)*600))
    m1y4.append(round((lcs[z][0][1][3]/1024)*600))
    m2y4.append(round((lcs[z][1][1][3]/1024)*600))
    m1x5.append(round((lcs[z][0][0][4]/1024)*600))
    m2x5.append(round((lcs[z][1][0][4]/1024)*600))
    m1y5.append(round((lcs[z][0][1][4]/1024)*600))
    m2y5.append(round((lcs[z][1][1][4]/1024)*600))
    m1x6.append(round((lcs[z][0][0][5]/1024)*600))
    m2x6.append(round((lcs[z][1][0][5]/1024)*600))
    m1y6.append(round((lcs[z][0][1][5]/1024)*600))
    m2y6.append(round((lcs[z][1][1][5]/1024)*600))
    m1x7.append(round((lcs[z][0][0][6]/1024)*600))
    m2x7.append(round((lcs[z][1][0][6]/1024)*600))
    m1y7.append(round((lcs[z][0][1][6]/1024)*600))
    m2y7.append(round((lcs[z][1][1][6]/1024)*600))
  return m1x1,m1x2,m1x3,m1x4,m1x5,m1x6,m1x7,m2x1,m2x2,m2x3,m2x4,m2x5,m2x6,m2x7,m1y1,m1y2,m1y3,m1y4,m1y5,m1y6,m1y7,m2y1,m2y2,m2y3,m2y4,m2y5,m2y6,m2y7        

In [15]:
def main(ch):
  rvpath = os.path.join(os.getcwd(),'task1_videos_mp4',ch)
  wvpath = os.path.join(os.getcwd(),'Images',ch)
  fn,ln = json_file(ch)
  frms,vids=video_process(rvpath,wvpath,ch,ln)
  vids.sort()
  frames = np.concatenate(frms, axis=0)
  bhvs,lcs = json_file_process(fn,vids)
  m1x1,m1x2,m1x3,m1x4,m1x5,m1x6,m1x7,m2x1,m2x2,m2x3,m2x4,m2x5,m2x6,m2x7,m1y1,m1y2,m1y3,m1y4,m1y5,m1y6,m1y7,m2y1,m2y2,m2y3,m2y4,m2y5,m2y6,m2y7 = parameter_extraction(lcs)
  df=pd.DataFrame()
  df['image'] = np.array(frames)
  df['m1x1'] = np.array(m1x1)
  df['m1y1'] = np.array(m1y1)
  df['m1x2'] = np.array(m1x2)
  df['m1y2'] = np.array(m1y2)
  df['m1x3'] = np.array(m1x3)
  df['m1y3'] = np.array(m1y3)
  df['m1x4'] = np.array(m1x4)
  df['m1y4'] = np.array(m1y4)
  df['m1x5'] = np.array(m1x5)
  df['m1y5'] = np.array(m1y5)
  df['m1x6'] = np.array(m1x6)
  df['m1y6'] = np.array(m1y6)
  df['m1x7'] = np.array(m1x7) 
  df['m1y7'] = np.array(m2y7)
  df['m2x1'] = np.array(m2x1)
  df['m2y1'] = np.array(m2y1)
  df['m2x2'] = np.array(m2x2)
  df['m2y2'] = np.array(m2y2)
  df['m2x3'] = np.array(m2x3)
  df['m2y3'] = np.array(m2y3)
  df['m2x4'] = np.array(m2x4)
  df['m2y4'] = np.array(m2y4)
  df['m2x5'] = np.array(m2x5)
  df['m2y5'] = np.array(m2y5)
  df['m2x6'] = np.array(m2x6)
  df['m2y6'] = np.array(m2y6)
  df['m2x7'] = np.array(m2x7) 
  df['m2y7'] = np.array(m2y7)
  df['behaviour'] = np.array(bhvs)
  return df


In [16]:
train_df = main('train') 

Video frames for the video mouse001_task1_annotator1.mp4 is processed
Video frames for the video mouse002_task1_annotator1.mp4 is processed
Video frames for the video mouse003_task1_annotator1.mp4 is processed
Video frames for the video mouse004_task1_annotator1.mp4 is processed
Video frames for the video mouse005_task1_annotator1.mp4 is processed
Video frames for the video mouse006_task1_annotator1.mp4 is processed
Video frames for the video mouse007_task1_annotator1.mp4 is processed
Video frames for the video mouse008_task1_annotator1.mp4 is processed
Video frames for the video mouse009_task1_annotator1.mp4 is processed
Video frames for the video mouse010_task1_annotator1.mp4 is processed
Video frames for the video mouse011_task1_annotator1.mp4 is processed
Video frames for the video mouse012_task1_annotator1.mp4 is processed
Video frames for the video mouse013_task1_annotator1.mp4 is processed
Video frames for the video mouse014_task1_annotator1.mp4 is processed
Video frames for the

In [17]:
train_df

,image,m1x1,m1y1,m1x2,m1y2,m1x3,m1y3,m1x4,m1y4,m1x5,...,m2y3,m2x4,m2y4,m2x5,m2y5,m2x6,m2y6,m2x7,m2y7,behaviour
0,mouse001_frameno_1.jpg,487,119,472,147,454,111,457,132,417,...,113,508,105,467,89,493,60,449,57,3
1,mouse001_frameno_2.jpg,488,118,474,148,456,114,459,135,424,...,118,510,110,469,90,496,62,449,58,1
2,mouse001_frameno_3.jpg,491,105,479,143,455,114,462,132,428,...,121,509,114,469,88,504,66,455,59,1
3,mouse001_frameno_4.jpg,484,103,478,138,454,110,460,128,436,...,121,502,113,466,86,502,66,461,57,1
4,mouse001_frameno_5.jpg,482,102,476,130,450,105,456,124,439,...,118,497,112,463,84,505,71,465,56,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154589,mouse018_frameno_6123.jpg,446,249,484,248,467,275,485,266,514,...,119,121,112,87,88,131,69,92,63,3
154590,mouse018_frameno_6124.jpg,451,268,473,254,471,276,484,268,501,...,121,121,113,88,86,131,69,93,62,3
154591,mouse018_frameno_6125.jpg,455,265,475,250,471,277,483,268,506,...,122,118,114,88,87,130,69,93,63,3
154592,mouse018_frameno_6126.jpg,459,261,490,249,485,281,495,267,516,...,124,117,115,89,87,133,72,95,63,3


In [18]:
train_df.to_csv('Train.csv',index=False )

In [19]:
test_df = main('test')

Video frames for the video mouse071_task1_annotator1.mp4 is processed
Video frames for the video mouse072_task1_annotator1.mp4 is processed
All video frame have been processed sucessfully!


In [20]:
test_df

,image,m1x1,m1y1,m1x2,m1y2,m1x3,m1y3,m1x4,m1y4,m1x5,...,m2y3,m2x4,m2y4,m2x5,m2y5,m2x6,m2y6,m2x7,m2y7,behaviour
0,mouse071_frameno_1.jpg,422,192,384,184,410,150,388,160,323,...,255,498,256,458,237,487,204,445,200,3
1,mouse071_frameno_2.jpg,430,194,389,188,413,153,392,164,325,...,257,496,259,457,241,488,208,446,203,1
2,mouse071_frameno_3.jpg,435,193,393,193,415,156,396,167,328,...,259,498,261,458,243,488,211,448,206,1
3,mouse071_frameno_4.jpg,440,195,398,196,419,159,399,170,333,...,259,499,259,459,245,489,212,449,208,1
4,mouse071_frameno_5.jpg,442,198,403,198,423,162,403,172,338,...,256,496,256,458,244,489,213,451,209,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41969,mouse072_frameno_18160.jpg,133,132,86,129,113,97,94,108,57,...,269,500,263,484,232,524,236,511,216,3
41970,mouse072_frameno_18161.jpg,126,136,84,125,115,99,95,106,57,...,269,499,262,483,231,524,236,510,215,3
41971,mouse072_frameno_18162.jpg,121,140,80,124,115,103,94,107,57,...,269,499,263,483,232,524,236,509,215,3
41972,mouse072_frameno_18163.jpg,116,144,78,123,115,106,92,108,58,...,269,499,262,483,230,524,236,510,215,3


In [21]:
test_df.to_csv('Val.csv',index=False ) 